In [0]:
dbutils.widgets.text("v_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("v_file_date")

In [0]:
%run "../includes/common_code"

Using race_results from presentation layer to get constructor standings

In [0]:
race_results_df = spark.read.parquet(f"{presentation_container_path}/race_results").filter(f"file_date = '{v_file_date}'")

In [0]:
from pyspark.sql.functions import col, when, count, sum, desc, asc

First way to get constructor standings but this don't has rank column but shows the correct order

In [0]:
constructor_standings_1 = race_results_df \
    .groupBy(race_results_df.race_year, race_results_df.team_name) \
        .agg(sum(col('points')).alias('points'), count(when(col('position') == 1, True)).alias('Wins')) \
            .orderBy(desc(col('points')), desc(col('Wins')))

In [0]:
display(constructor_standings_1.filter(constructor_standings_1.race_year == 2020))

race_year,team_name,points,Wins


Lets do it using window functions, with this you can get rank

In [0]:
from pyspark.sql import Window
from pyspark.sql.functions import rank

In [0]:
constructor_standings_2 = race_results_df \
    .groupBy(race_results_df.race_year, race_results_df.team_name) \
        .agg(sum(col('points')).alias('points'), count(when(col('position') == 1, True)).alias('Wins'))

In [0]:
constructor_rank = Window.partitionBy('race_year').orderBy(desc(col('points')), desc(col('Wins')))
constructor_standings_2 = constructor_standings_2.withColumn('rank', rank().over(constructor_rank))

In [0]:
# Lets reorder according to the BBC driver standings page
constructor_standings_final = constructor_standings_2.select(constructor_standings_2.rank, constructor_standings_2.team_name, constructor_standings_2.Wins, constructor_standings_2.points, constructor_standings_2.race_year)

In [0]:
display(constructor_standings_final.filter(constructor_standings_final.race_year == 2020))

rank,team_name,Wins,points,race_year


In [0]:
# Lets write this into presentation layer
overwrite_partition(constructor_standings_final, 'f1_presentation', 'constructor_standings', 'race_year')

In [0]:
dbutils.notebook.exit("success")